# Sarcastic Detection MLOps

In [116]:
!pip install tfx tensorflow_model_analysis

## Import Library

In [117]:
#import library
import os
import pandas as pd
import tensorflow_model_analysis as tfma
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator
from tfx.components import Transform, Trainer, Tuner, Evaluator, Pusher
from tfx.proto import example_gen_pb2, trainer_pb2, pusher_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing

## Set Variabel

In [118]:
PIPELINE_NAME = "daffamh-pipeline"
SCHEMA_PIPELINE_NAME = "daffamh-schema"

PIPELINE_ROOT = PIPELINE_NAME

METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')

SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

### Inisiasi data root

In [119]:
DATA_ROOT = 'data'

## Load Dataset

In [120]:
sarcastic_df = pd.read_csv("headlines_dataset.csv")
sarcastic_df.head()

headline  \
0  Ook mysterieuze metalen monoliet opgedoken bij het Friese dorpje Oudehorne   
1    Politie doet onderzoek naar racistische bekladding van pand in Amsterdam   
2                Rondvliegende kogel raakt muur boven bed van 87-jarige vrouw   
3      Weekweerbericht: Koude week, in Limburg maandag kans op (natte) sneeuw   
4        Weerbericht: Ochtend begint met lichte vorst, later zonnige perioden   

                                                                                                               link  \
0  https://nu.nl/binnenland/6094927/ook-mysterieuze-metalen-monoliet-opgedoken-bij-het-friese-dorpje-oudehorne.html   
1    https://nu.nl/binnenland/6094921/politie-doet-onderzoek-naar-racistische-bekladding-van-pand-in-amsterdam.html   
2                https://nu.nl/binnenland/6094897/rondvliegende-kogel-raakt-muur-boven-bed-van-87-jarige-vrouw.html   
3          https://nu.nl/binnenland/6094881/weekweerbericht-koude-week-in-limburg-maandag-kans-op-natte-sneeuw.html   
4          https://nu.nl/binnenland/6094863/weerbericht-ochtend-begint-met-lichte-vorst-later-zonnige-perioden.html   

  source  is_sarcastic  is_binnenland  is_buitenland  is_politiek  
0  Nu.nl             0              1              0            0  
1  Nu.nl             0              1              0            0  
2  Nu.nl             0              1              0            0  
3  Nu.nl             0              1              0            0  
4  Nu.nl             0              1              0            0

In [121]:
sarcastic_df = sarcastic_df.drop(["link", "source", "is_binnenland", "is_buitenland", "is_politiek"], axis=1)
sarcastic_df.to_csv(os.path.join(DATA_ROOT, "news.csv"), index=False)

In [122]:
interactive_context = InteractiveContext(pipeline_root = PIPELINE_ROOT)

## Data Ingestion

### Melakukan data ingestion dengan module ExampleGen

In [123]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
    ])
)
example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

#### ExampleGen ini akan membagi dataset sebesar 80:20 untuk training dan validation

In [124]:
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 33
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

## Data Validation

### Menampilkan Summary Statistics dengan StatisticsGen

In [125]:
statistic_gen = StatisticsGen(
    examples = example_gen.outputs["examples"]
)

interactive_context.run(statistic_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 34
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

### Menampilkan statistics

In [126]:
interactive_context.show(statistic_gen.outputs['statistics'])

### Membuat Data Schema Dengan SchemaGen

In [127]:
schema_gen = SchemaGen(statistics = statistic_gen.outputs["statistics"])

interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 35
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

### Menampilkan SchemaGen

In [128]:
interactive_context.show(schema_gen.outputs["schema"])

Type  Presence Valency Domain
Feature name                                  
'headline'      BYTES  required              -
'is_sarcastic'    INT  required              -

### Mengidentifikasi Anomali dengan ExampleGen

In [129]:
example_validator = ExampleValidator(
    statistics = statistic_gen.outputs["statistics"],
    schema = schema_gen.outputs["schema"]
)

interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 36
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))


### Menampilkan Anomali

In [130]:
interactive_context.show(example_validator.outputs["anomalies"])

## Data Preprocessing

### Transform

### membuat sarcastic_transform untuk menyimpan proses transform

In [131]:
TRANSFORM_MODULE_FILE = "sarcastic_transform.py"

### Module stress_transform.py untuk mengubah nama fitur yang telah ditransform dengan menambahkan _xf, dan melakukan proses mengubah label menjadi integer dan text menjadi lowercase

In [132]:
# Create sarcastic_transform.py in the content directory
transform_code = """
import tensorflow as tf

LABEL_KEY = "is_sarcastic"
FEATURE_KEY = "headline"

def transformed_name(key):
    \"\"\"Renaming transformed features\"\"\"
    return key + "_xf"

def preprocessing_fn(inputs):
    \"\"\"
    Preprocess input features into transformed features

    Args:
        inputs: map from feature keys to raw features.

    Return:
        outputs: map from feature keys to transformed features.
    \"\"\"

    outputs = {}

    # Lowercase the headline text and handle missing strings
    outputs[transformed_name(FEATURE_KEY)] = tf.strings.lower(
        tf.where(
            tf.strings.length(inputs[FEATURE_KEY]) > 0,
            inputs[FEATURE_KEY],
            tf.constant("", dtype=tf.string)
        )
    )

    # Cast the 'label' to int64
    outputs[transformed_name(LABEL_KEY)] = tf.cast(inputs[LABEL_KEY], tf.int64)

    return outputs
"""

# Write the transform code to a Python file
with open("sarcastic_transform.py", "w") as f:
    f.write(transform_code)

### Melakukan transform

In [133]:
transform  = Transform(
    examples=example_gen.outputs['examples'],
    schema= schema_gen.outputs['schema'],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)


### Menampilkan transform

In [134]:
interactive_context.run(transform)

ExecutionResult(
    component_id: Transform
    execution_id: 37
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

## Trainer Hyperparameter


In [135]:
TRAINER_MODULE_FILE = "sarcastic_trainer.py"

### Menuliskan Module sarcastic_tuner.py untuk kebutuhan tuning

In [136]:
%%writefile {TRAINER_MODULE_FILE}
import tensorflow as tf
import tensorflow_transform as tft
from tensorflow.keras import layers
import os
import tensorflow_hub as hub
from tfx.components.trainer.fn_args_utils import FnArgs

LABEL_KEY = "is_sarcastic"
FEATURE_KEY = "headline"

def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"

def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')


def input_fn(file_pattern,
             tf_transform_output,
             num_epochs,
             shuffle,
             batch_size=64)->tf.data.Dataset:
    """Get post_tranform feature & create batches of data"""

    # Get post_transform feature spec
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy())

    # create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        shuffle=shuffle,
        label_key = transformed_name(LABEL_KEY))
    return dataset

# os.environ['TFHUB_CACHE_DIR'] = '/hub_chace'
# embed = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4")

# Vocabulary size and number of words in a sequence.
VOCAB_SIZE = 10000
SEQUENCE_LENGTH = 100

vectorize_layer = layers.TextVectorization(
    standardize="lower_and_strip_punctuation",
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=SEQUENCE_LENGTH)


embedding_dim=16
def model_builder():
    """Build machine learning model"""
    inputs = tf.keras.Input(shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string)
    reshaped_narrative = tf.reshape(inputs, [-1])
    x = vectorize_layer(reshaped_narrative)
    x = layers.Embedding(VOCAB_SIZE, embedding_dim, name="embedding")(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(16, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)


    model = tf.keras.Model(inputs=inputs, outputs = outputs)

    model.compile(
        loss = 'binary_crossentropy',
        optimizer=tf.keras.optimizers.Adam(0.001),
        metrics=['Accuracy','AUC','FalsePositives','TruePositives', 'FalseNegatives', 'TrueNegatives']

    )

    # print(model)
    model.summary()
    return model


def _get_serve_tf_examples_fn(model, tf_transform_output):

    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):

        feature_spec = tf_transform_output.raw_feature_spec()

        feature_spec.pop(LABEL_KEY)

        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)

        transformed_features = model.tft_layer(parsed_features)

        # get predictions using the transformed features
        return model(transformed_features)

    return serve_tf_examples_fn

def run_fn(fn_args: FnArgs) -> None:

    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')

    # TensorBoard callback for tracking metrics during training
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir, update_freq='batch'
    )

    # EarlyStopping callback to prevent overfitting
    es = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',  # Changed to monitor 'val_loss' since 'val_binary_accuracy' wasn't available
        mode='min',          # 'min' since we want to minimize the loss
        verbose=2,
        patience=10
    )

    # ModelCheckpoint to save the best model based on validation loss
    mc = tf.keras.callbacks.ModelCheckpoint(
        filepath=fn_args.serving_model_dir,
        monitor='val_loss',  # Changed to 'val_loss' since 'val_binary_accuracy' wasn't available
        mode='min',          # 'min' since we want to minimize the loss
        verbose=2,
        save_best_only=True
    )

    # Load the transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    # Create batches of data, ensure repeat() is called to avoid dataset exhaustion
    train_set = input_fn(fn_args.train_files, tf_transform_output, num_epochs=None, shuffle=True)
    val_set = input_fn(fn_args.eval_files, tf_transform_output, num_epochs=None, shuffle=False)
    training_steps = 19000 // 64
    validation_steps = 4000 // 64

    # Adapt the vectorize layer to the training data
    vectorize_layer.adapt(
        [j[0].numpy()[0] for j in [
            i[0][transformed_name(FEATURE_KEY)]
                for i in list(train_set.take(1000))]])  # Limit to 1000 samples for adaptation

    # Build the model
    model = model_builder()

    # Train the model
    model.fit(
        x=train_set,
        validation_data=val_set,
        callbacks=[tensorboard_callback, es, mc],
        steps_per_epoch=1000,           # Ensure you have enough data for 1000 steps
        validation_steps=validation_steps,          # Ensure enough validation data is available
        epochs=17
    )

    # Save the model with the proper signature
    signatures = {
        'serving_default':
        _get_serve_tf_examples_fn(model, tf_transform_output).get_concrete_function(
            tf.TensorSpec(
                shape=[None],
                dtype=tf.string,
                name='examples'))
    }
    model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

Overwriting sarcastic_trainer.py


In [137]:
from tfx.proto import trainer_pb2

trainer  = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    examples = transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train']),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'])
)
interactive_context.run(trainer)

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 headline_xf (InputLayer)    [(None, 1)]               0         
                                                                 
 tf.reshape_3 (TFOpLambda)   (None,)                   0         
                                                                 
 text_vectorization_3 (Text  (None, 100)               0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 100, 16)           160000    
                                                                 
 global_average_pooling1d_3  (None, 16)                0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_9 (Dense)             (None, 64)                1088

ExecutionResult(
    component_id: Trainer
    execution_id: 38
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

## Analisis dan Evaluasi Model

#### Menganalisa dan mengevaluasi mode  dengan menggunakan resolver untuk menguji beberapa metric beserta nilai threshold pada metric tersebut

In [147]:
model_resolver = Resolver(
    strategy_class=LatestBlessedModelStrategy,
    model=Channel(type=Model),
    model_blessing=Channel(type=ModelBlessing),
).with_id("Latest_blessed_model_resolver")

interactive_context.run(model_resolver)


ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 42
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [148]:
eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key="is_sarcastic")],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            tfma.MetricConfig(class_name="AUC"),
            tfma.MetricConfig(class_name="TruePositives"),
            tfma.MetricConfig(class_name="FalsePositives"),
            tfma.MetricConfig(class_name="TrueNegatives"),
            tfma.MetricConfig(class_name="FalseNegatives"),
            tfma.MetricConfig(class_name="BinaryAccuracy",
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={"value": 0.6},
                    ),
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={"value": 1e-4},
                    ),
                ),
            ),
        ])
    ]
)

### Menampilkan hasil evaluator

In [140]:
evaluator = Evaluator(
    examples=example_gen.outputs["examples"],
    model=trainer.outputs["model"],
    baseline_model=model_resolver.outputs["model"],
    eval_config=eval_config,
)
interactive_context.run(evaluator)

ExecutionResult(
    component_id: Evaluator
    execution_id: 40
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [141]:
eval_result = evaluator.outputs["evaluation"].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(tfma_result)

## Export Model

In [142]:
pusher = Pusher(
    model=trainer.outputs["model"],
    model_blessing=evaluator.outputs["blessing"],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=os.path.join(
                SERVING_MODEL_DIR, "deutch-sarcastic-detection-model"
            ),
        )
    )
)

interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 41
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

## Compress folder pipelines dan serving_model


In [143]:
!zip -r /content/daffa-pipeline.zip /content/sarcastic-pipeline


	zip warning: name not matched: /content/sarcastic-pipeline

zip error: Nothing to do! (try: zip -r /content/daffa-pipeline.zip . -i /content/sarcastic-pipeline)


In [144]:
!zip -r /content/serving_model_dir.zip /content/serving_model_dir/

	zip warning: name not matched: /content/serving_model_dir/

zip error: Nothing to do! (try: zip -r /content/serving_model_dir.zip . -i /content/serving_model_dir/)


In [145]:
!pip freeze >> requirements.txt

In [146]:
!zip -r /content/file.zip /content/

updating: content/ (stored 0%)
updating: content/.config/ (stored 0%)
updating: content/.config/configurations/ (stored 0%)
updating: content/.config/configurations/config_default (deflated 15%)
updating: content/.config/config_sentinel (stored 0%)
updating: content/.config/hidden_gcloud_config_universe_descriptor_data_cache_configs.db (deflated 97%)
updating: content/.config/active_config (stored 0%)
updating: content/.config/gce (stored 0%)
updating: content/.config/.last_update_check.json (deflated 24%)
updating: content/.config/default_configs.db (deflated 98%)
updating: content/.config/.last_survey_prompt.yaml (stored 0%)
updating: content/.config/.last_opt_in_prompt.yaml (stored 0%)
updating: content/.config/logs/ (stored 0%)
updating: content/.config/logs/2024.10.02/ (stored 0%)
updating: content/.config/logs/2024.10.02/13.25.16.787495.log (deflated 58%)
updating: content/.config/logs/2024.10.02/13.25.40.484117.log (deflated 57%)
updating: content/.config/logs/2024.10.02/13.25.4